In [83]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda
import os
import pandas as pd
from torch.utils.data import Dataset
import numpy


df = pd.read_csv("10SecondStepPriceEvolution2.csv")
df["F10"] = (df["F10"] - df["Cur"]).astype('float64')
df["P10"] = (df["P10"] - df["Cur"]).astype('float64')
df["P20"] = (df["P20"] - df["Cur"]).astype('float64')
df["P30"] = (df["P30"] - df["Cur"]).astype('float64')
df["P40"] = (df["P40"] - df["Cur"]).astype('float64')
df["P50"] = (df["P50"] - df["Cur"]).astype('float64')
df["P60"] = (df["P60"] - df["Cur"]).astype('float64')
df.drop('Cur', axis=1, inplace=True)
#df['F10'] = df['F10'].apply(lambda x: 1 if x >= 1 else -1 if x <= -1 else 0)
#zeros = df[df['F10'] == 0]
#ones  = df[df['F10'] == 1]
#negas = df[df['F10'] == -1]
#zeros = zeros.sample(frac=1).reset_index(drop=True)
#zeros = zeros[:15000]
#df = pd.concat([zeros, ones, negas])
#df = df.sample(frac=1).reset_index(drop=True)
train_dataset = df.head(60000)
test_dataset = df.tail(df.shape[0] - 60000)


class CustomPriceDataset(Dataset):
    def __init__(self, dataframe, labels, transform=None, target_transform=None):
        self.dataframe = dataframe
        self.labels = labels
        self.transform = transform
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = list(self.dataframe.iloc[idx])
        label = self.labels.iloc[idx]
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label


trnlbl = train_dataset['F10'] + 1
trnset = train_dataset.drop('F10', axis=1)
trainPrice = CustomPriceDataset(trnset, trnlbl, torch.FloatTensor)
train_dataloader = DataLoader(trainPrice, batch_size=64)

tstlbl = test_dataset['F10'] + 1
tstset = test_dataset.drop('F10', axis=1)
testPrice = CustomPriceDataset(tstset, tstlbl, torch.FloatTensor)
test_dataloader = DataLoader(testPrice, batch_size=64)

In [84]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(6, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [85]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        print(loss.type())
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [86]:
learning_rate = 1e-2
passed = 0

In [87]:
epochs = 5
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=1e-2, dampening=1e-3)

In [88]:
print("Start")
for t in range(epochs):
    print(f"Epoch {t+1 + passed}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
passed = passed + epochs
print("Done!")

Start
Epoch 1
-------------------------------
torch.DoubleTensor


/home/vikcoc/anaconda3/envs/learnMl/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: Found dtype Double but expected Float